In [2]:
from gensim.models import KeyedVectors

In [10]:
word_vectors = KeyedVectors.load_word2vec_format('bow2x.words', binary=False) 

In [9]:
with open('bow2.words', 'r') as f:
    text = f.read()
    
text = '183870 300\n' + text

with open('bow2x.words', 'w+') as f:
    f.write(text)

In [11]:
print(word_vectors)